In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


# Primeros pasos
usar rol AccountAdmin para tener suficientes permisos para todo el proceso e inicialmente crear los siguientes objetos:

- Base de datos "DEMO_VENTAS_VEHICULOS".
- Esquemas RAW, REFINED, CURATED dentro de la base de datos.

In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
USE ROLE ACCOUNTADMIN;
CREATE OR REPLACE DATABASE DEMO_VENTA_VEHICULOS;
CREATE OR REPLACE SCHEMA DEMO_VENTA_VEHICULOS.RAW;
CREATE OR REPLACE SCHEMA DEMO_VENTA_VEHICULOS.REFINED;
CREATE OR REPLACE SCHEMA DEMO_VENTA_VEHICULOS.CURATED;

In [ ]:
CREATE OR REPLACE STAGE DEMO_VENTA_VEHICULOS.RAW.MANUALES
    DIRECTORY = (ENABLE = TRUE)
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

Descarga los dos manuales de ejemplo disponibles aquí:

- [Nissan Qashqai Guía de Referencia Rápida](https://manualesdemocortexvehiculos.s3.eu-south-2.amazonaws.com/Nissan_Qashqai_Guia_de_Referencia_Rapida.pdf)
- [Nissan Qashqai Manual del Conductor](https://manualesdemocortexvehiculos.s3.eu-south-2.amazonaws.com/Nissan_Qashqai_Manual_del_Conductor.pdf)

Se han eliminado algunas páginas del manual del conductor para que no pasen de 500 páginas, ya que actualmente la función de AISQL tiene un límite que no permite parsear documentos de más de 500 páginas.

A continuación, sube ambos PDFs dede la UI de Snowflake (Snowsight) al STAGE creado en la celda anterior que se encuentra en:

- Base de datos: DEMO_VENTA_VEHICULOS
- Esquema: RAW
- Nombre del Stage: MANUALES

# Cargar datos desde S3
A continuación se cargarán todos los microdatos de matriculaciones de vehículos de la Dirección General de Tráfico desde un bucket de S3 de acceso público.

Los datos originales están disponibles en: https://www.dgt.es/menusecundario/dgt-en-cifras/dgt-en-cifras-resultados/dgt-en-cifras-detalle/Microdatos-de-Matriculaciones-de-Vehiculos-mensual/

El proceso para cargar los datos es:

- Crear un FILE FORMAT para que se carguen los datos correctamente. Los datos están en ficheros planos txt sin separador de campos y con un salto de línea para cada registro y con la primera fila de información no útil para el análisis. A partir de la fila 2, cada registro representa una matriculación de un vehículo.
- A continuación se crea la tabla DEMO_VENTA_VEHICULOS.RAW.CSV_MATRICULACIONES donde se copiarán todos los datos RAW.
- Finalmente se copian todos los datos de los ficheros en S3 a dicha tabla.

In [ ]:
-- Crear File Format para leer correctamente los datos RAW
CREATE OR REPLACE FILE FORMAT DEMO_VENTA_VEHICULOS.RAW.CSV_SINGLE_COLUMN
    TYPE = 'CSV'
    FIELD_DELIMITER = NONE
    RECORD_DELIMITER = '\n'
    SKIP_HEADER = 1
    ENCODING = 'ISO88591';

-- Crear tabla para insertar datos RAW
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.RAW.CSV_MATRICULACIONES
(RAW VARCHAR);

-- Insertar datos de S3 en tabla RAW
COPY INTO DEMO_VENTA_VEHICULOS.RAW.CSV_MATRICULACIONES
FROM 's3://dgtmicrodatosmatriculacionesmensuales/'
FILE_FORMAT = ( FORMAT_NAME = DEMO_VENTA_VEHICULOS.RAW.CSV_SINGLE_COLUMN)
ON_ERROR = 'skip_file';

In [ ]:
-- Comprobar datos cargados
SELECT * FROM DEMO_VENTA_VEHICULOS.RAW.CSV_MATRICULACIONES LIMIT 5;

# Dividir campos de matriculaciones y tablas de referencia
En este largo apartado vamos a crear una tabla en el esquema REFINED donde dividiremos los datos RAW de matriculaciones en las distintas columnas correspondientes.

Además, crearemos tablas de referencia para poder pasar de códigos a valores descriptivos en varias variables e insertaremos los valores correspondientes.

Toda esta información está además disponible en el documento PDF de diseño de registro de la DGT disponible en https://www.dgt.es/export/sites/web-DGT/.galleries/downloads/dgt-en-cifras/matraba/MATRICULACIONES_MATRABA.pdf

In [ ]:
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.FACT_MATRICULACIONES (
    FECHA_MATRICULA VARCHAR, 
    COD_CLASE_MAT VARCHAR, 
    FEC_TRAMITACION VARCHAR, 
    MARCA_ITV VARCHAR, 
    MODELO_ITV VARCHAR, 
    COD_PROCEDENCIA_ITV VARCHAR,
    BASTIDOR_ITV VARCHAR,
    COD_TIPO VARCHAR,
    COD_PROPULSION_ITV VARCHAR,
    CILINDRADA_ITV VARCHAR,
    POTENCIA_ITV VARCHAR,
    TARA VARCHAR,
    PESO_MAX VARCHAR,
    NUM_PLAZAS VARCHAR,
    IND_PRECINTO VARCHAR,
    IND_EMBARGO VARCHAR,
    NUM_TRANSMISIONES VARCHAR,
    NUM_TITULARES VARCHAR,
    LOCALIDAD_VEHICULO VARCHAR,
    COD_PROVINCIA_VEH VARCHAR,
    COD_PROVINCITA_MAT VARCHAR,
    CLAVE_TRAMITE VARCHAR,
    FEC_TRAMITE VARCHAR,
    CODIGO_POSTAL VARCHAR,
    FEC_PRIM_MATRICULACION VARCHAR,
    IND_NUEVO_USADO VARCHAR,
    PERSONA_FISICA_JURIDICA VARCHAR,
    CODIGO_ITV VARCHAR,
    SERVICIO VARCHAR,
    COD_MUNICIPIO_INE_VEH VARCHAR,
    MUNICIPIO VARCHAR,
    KW_ITV VARCHAR,
    NUM_PLAZAS_MAX VARCHAR,
    CO2_ITV VARCHAR,
    RENTING VARCHAR,
    COD_TUTELA VARCHAR,
    COD_POSESION VARCHAR,
    IND_BAJA_DEF VARCHAR,
    IND_BAJA_TEMP VARCHAR,
    IND_SUSTRACCION VARCHAR,
    BAJA_TELEMATICA VARCHAR,
    TIPO_ITV VARCHAR,
    VARIANTE_ITV VARCHAR,
    VERSION_ITV VARCHAR,
    FABRICANTE_ITV VARCHAR,
    MASA_ORDEN_MARCHA_ITV VARCHAR,
    MASA_MAXIMA_TECNICA_ADMISIBLE VARCHAR,
    CATEGORIA_HOMOLOGACION_EUROPEA_ITV VARCHAR,
    CARROCERIA VARCHAR,
    PLAZAS_PIE VARCHAR,
    NIVEL_EMISIONES_EURO_ITV VARCHAR,
    CONSUMO_WHKM_ITV VARCHAR,
    CLASIFICACION_REGLAMENTO_VEHICULOS_ITV VARCHAR,
    CATEGORIA_VEHICULO_ELECTRICO VARCHAR,
    AUTONOMIA_VEHICULO_ELECTRICO VARCHAR,
    MARCA_VEHICULO_BASE VARCHAR,
    FABRICANTE_VEHICULO_BASE VARCHAR,
    TIPO_VEHICULO_BASE VARCHAR,
    VARIANTE_VEHICULO_BASE VARCHAR,
    VERSION_VEHICULO_BASE VARCHAR,
    DISTANCIA_EJES_12_ITV VARCHAR,
    VIA_ANTERIOR_ITV VARCHAR,
    VIA_POSTERIOR_ITV VARCHAR,
    TIPO_ALIMENTACION_ITV VARCHAR,
    CONTRASENA_HOMOLOGACION_ITV VARCHAR,
    ECO_INNOVACION_ITV VARCHAR,
    REDUCCION_ECO_ITV VARCHAR,
    CODIGO_ECO_ITV VARCHAR,
    FEC_PROCESO VARCHAR    
    ) AS 
SELECT
    CASE WHEN TRIM(SUBSTR(RAW,1,8))='' THEN NULL ELSE TRIM(SUBSTR(RAW,0,8)) END,
    CASE WHEN TRIM(SUBSTR(RAW,9,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,9,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,10,8))='' THEN NULL ELSE TRIM(SUBSTR(RAW,10,8)) END,
    CASE WHEN TRIM(SUBSTR(RAW,18,30))='' THEN NULL ELSE TRIM(SUBSTR(RAW,18,30)) END,
    CASE WHEN TRIM(SUBSTR(RAW,48,22))='' THEN NULL ELSE TRIM(SUBSTR(RAW,48,22)) END,
    CASE WHEN TRIM(SUBSTR(RAW,70,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,70,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,71,21))='' THEN NULL ELSE TRIM(SUBSTR(RAW,71,21)) END,
    CASE WHEN TRIM(SUBSTR(RAW,92,2))='' THEN NULL ELSE TRIM(SUBSTR(RAW,92,2)) END,
    CASE WHEN TRIM(SUBSTR(RAW,94,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,94,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,95,5))='' THEN NULL ELSE TRIM(SUBSTR(RAW,95,5)) END,
    CASE WHEN TRIM(SUBSTR(RAW,100,6))='' THEN NULL ELSE TRIM(SUBSTR(RAW,100,6)) END,
    CASE WHEN TRIM(SUBSTR(RAW,106,6))='' THEN NULL ELSE TRIM(SUBSTR(RAW,106,6)) END,
    CASE WHEN TRIM(SUBSTR(RAW,112,6))='' THEN NULL ELSE TRIM(SUBSTR(RAW,112,6)) END,
    CASE WHEN TRIM(SUBSTR(RAW,118,3))='' THEN NULL ELSE TRIM(SUBSTR(RAW,118,3)) END,
    CASE WHEN TRIM(SUBSTR(RAW,121,2))='' THEN NULL ELSE TRIM(SUBSTR(RAW,121,2)) END,
    CASE WHEN TRIM(SUBSTR(RAW,123,2))='' THEN NULL ELSE TRIM(SUBSTR(RAW,123,2)) END,
    CASE WHEN TRIM(SUBSTR(RAW,125,2))='' THEN NULL ELSE TRIM(SUBSTR(RAW,125,2)) END,
    CASE WHEN TRIM(SUBSTR(RAW,127,2))='' THEN NULL ELSE TRIM(SUBSTR(RAW,127,2)) END,
    CASE WHEN TRIM(SUBSTR(RAW,129,24))='' THEN NULL ELSE TRIM(SUBSTR(RAW,129,24)) END,
    CASE WHEN TRIM(SUBSTR(RAW,153,2))='' THEN NULL ELSE TRIM(SUBSTR(RAW,153,2)) END,
    CASE WHEN TRIM(SUBSTR(RAW,155,2))='' THEN NULL ELSE TRIM(SUBSTR(RAW,155,2)) END,
    CASE WHEN TRIM(SUBSTR(RAW,157,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,157,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,158,8))='' THEN NULL ELSE TRIM(SUBSTR(RAW,158,8)) END,
    CASE WHEN TRIM(SUBSTR(RAW,166,5))='' THEN NULL ELSE TRIM(SUBSTR(RAW,166,5)) END,
    CASE WHEN TRIM(SUBSTR(RAW,171,8))='' THEN NULL ELSE TRIM(SUBSTR(RAW,171,8)) END,
    CASE WHEN TRIM(SUBSTR(RAW,179,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,179,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,180,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,180,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,181,9))='' THEN NULL ELSE TRIM(SUBSTR(RAW,181,9)) END,
    CASE WHEN TRIM(SUBSTR(RAW,190,3))='' THEN NULL ELSE TRIM(SUBSTR(RAW,190,3)) END,
    CASE WHEN TRIM(SUBSTR(RAW,193,5))='' THEN NULL ELSE TRIM(SUBSTR(RAW,193,5)) END,
    CASE WHEN TRIM(SUBSTR(RAW,198,30))='' THEN NULL ELSE TRIM(SUBSTR(RAW,198,30)) END,
    CASE WHEN TRIM(SUBSTR(RAW,228,7))='' THEN NULL ELSE TRIM(SUBSTR(RAW,228,7)) END,
    CASE WHEN TRIM(SUBSTR(RAW,235,3))='' THEN NULL ELSE TRIM(SUBSTR(RAW,235,3)) END,
    CASE WHEN TRIM(SUBSTR(RAW,238,5))='' THEN NULL ELSE TRIM(SUBSTR(RAW,238,5)) END,
    CASE WHEN TRIM(SUBSTR(RAW,243,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,243,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,244,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,244,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,245,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,245,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,246,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,246,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,247,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,247,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,248,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,248,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,249,11))='' THEN NULL ELSE TRIM(SUBSTR(RAW,249,11)) END,
    CASE WHEN TRIM(SUBSTR(RAW,260,25))='' THEN NULL ELSE TRIM(SUBSTR(RAW,260,25)) END,
    CASE WHEN TRIM(SUBSTR(RAW,285,25))='' THEN NULL ELSE TRIM(SUBSTR(RAW,285,25)) END,
    CASE WHEN TRIM(SUBSTR(RAW,310,35))='' THEN NULL ELSE TRIM(SUBSTR(RAW,310,35)) END,
    CASE WHEN TRIM(SUBSTR(RAW,345,70))='' THEN NULL ELSE TRIM(SUBSTR(RAW,345,70)) END,
    CASE WHEN TRIM(SUBSTR(RAW,415,6))='' THEN NULL ELSE TRIM(SUBSTR(RAW,415,6)) END,
    CASE WHEN TRIM(SUBSTR(RAW,421,6))='' THEN NULL ELSE TRIM(SUBSTR(RAW,421,6)) END,
    CASE WHEN TRIM(SUBSTR(RAW,427,4))='' THEN NULL ELSE TRIM(SUBSTR(RAW,427,4)) END,
    CASE WHEN TRIM(SUBSTR(RAW,431,4))='' THEN NULL ELSE TRIM(SUBSTR(RAW,431,4)) END,
    CASE WHEN TRIM(SUBSTR(RAW,435,3))='' THEN NULL ELSE TRIM(SUBSTR(RAW,435,3)) END,
    CASE WHEN TRIM(SUBSTR(RAW,438,8))='' THEN NULL ELSE TRIM(SUBSTR(RAW,438,8)) END,
    CASE WHEN TRIM(SUBSTR(RAW,446,4))='' THEN NULL ELSE TRIM(SUBSTR(RAW,446,4)) END,
    CASE WHEN TRIM(SUBSTR(RAW,450,4))='' THEN NULL ELSE TRIM(SUBSTR(RAW,450,4)) END,
    CASE WHEN TRIM(SUBSTR(RAW,454,4))='' THEN NULL ELSE TRIM(SUBSTR(RAW,454,4)) END,
    CASE WHEN TRIM(SUBSTR(RAW,458,6))='' THEN NULL ELSE TRIM(SUBSTR(RAW,458,6)) END,
    CASE WHEN TRIM(SUBSTR(RAW,464,30))='' THEN NULL ELSE TRIM(SUBSTR(RAW,464,30)) END,
    CASE WHEN TRIM(SUBSTR(RAW,494,50))='' THEN NULL ELSE TRIM(SUBSTR(RAW,494,50)) END,
    CASE WHEN TRIM(SUBSTR(RAW,544,35))='' THEN NULL ELSE TRIM(SUBSTR(RAW,544,35)) END,
    CASE WHEN TRIM(SUBSTR(RAW,579,25))='' THEN NULL ELSE TRIM(SUBSTR(RAW,579,25)) END,
    CASE WHEN TRIM(SUBSTR(RAW,604,35))='' THEN NULL ELSE TRIM(SUBSTR(RAW,604,35)) END,
    CASE WHEN TRIM(SUBSTR(RAW,639,4))='' THEN NULL ELSE TRIM(SUBSTR(RAW,639,4)) END,
    CASE WHEN TRIM(SUBSTR(RAW,643,4))='' THEN NULL ELSE TRIM(SUBSTR(RAW,643,4)) END,
    CASE WHEN TRIM(SUBSTR(RAW,647,4))='' THEN NULL ELSE TRIM(SUBSTR(RAW,647,4)) END,
    CASE WHEN TRIM(SUBSTR(RAW,651,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,651,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,652,25))='' THEN NULL ELSE TRIM(SUBSTR(RAW,652,25)) END,
    CASE WHEN TRIM(SUBSTR(RAW,677,1))='' THEN NULL ELSE TRIM(SUBSTR(RAW,677,1)) END,
    CASE WHEN TRIM(SUBSTR(RAW,678,4))='' THEN NULL ELSE TRIM(SUBSTR(RAW,678,4)) END,
    CASE WHEN TRIM(SUBSTR(RAW,682,25))='' THEN NULL ELSE TRIM(SUBSTR(RAW,682,25)) END,
    CASE WHEN TRIM(SUBSTR(RAW,707,8))='' THEN NULL ELSE TRIM(SUBSTR(RAW,707,8)) END
FROM
  DEMO_VENTA_VEHICULOS.RAW.CSV_MATRICULACIONES;

CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_CLASE_MAT (COD_CLASE_MAT VARCHAR, DESCRIPCION VARCHAR);

INSERT INTO DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_CLASE_MAT VALUES
    ('0','Ordinaria'),
    ('1','Turistica'),
    ('2','Remolque'),
    ('3','Diplomatica'),
    ('4','Reservada'),
    ('5','Vehiculo especial'),
    ('6','Ciclomotor'),
    ('7','Transporte temporal'),
    ('8','Historica');

CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_PROCEDENCIA (COD_PROCEDENCIA VARCHAR, DESCRIPCION VARCHAR);

INSERT INTO DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_PROCEDENCIA VALUES
    ('0','Fabricacion nacional'),
    ('1','Importacion no comunitaria'),
    ('2','Subasta'),
    ('3','Importacion UE');
    
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_TIPO (COD_TIPO VARCHAR, DESCRIPCION VARCHAR);

INSERT INTO DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_TIPO VALUES
    ('00', 'CAMIÓN'),
    ('01', 'CAMIÓN PLATAFORMA'),
    ('02', 'CAMIÓN CAJA'),
    ('03', 'CAMIÓN FURGÓN'),
    ('04', 'CAMIÓN BOTELLERO'),
    ('05', 'CAMIÓN CISTERNA'),
    ('06', 'CAMIÓN JAULA'),
    ('07', 'CAMIÓN FRIGORÍFICO'),
    ('08', 'CAMIÓN TALLER'),
    ('09', 'CAMIÓN PARA CANTERA'),
    ('0A', 'CAMIÓN PORTAVEHÍCULOS'),
    ('0B', 'CAMIÓN MIXTO'),
    ('0C', 'CAMIÓN PORTACONTENEDORES'),
    ('0D', 'CAMIÓN BASURERO'),
    ('0E', 'CAMIÓN ISOTERMO'),
    ('0F', 'CAMIÓN SILO'),
    ('0G', 'VEHICULO MIXTO ADAPTABLE'),
    ('10', 'CAMIÓN ARTICULADO'),
    ('11', 'CAMIÓN ARTICULADO PLATAFORMA'),
    ('12', 'CAMIÓN ARTICULADO CAJA'),
    ('13', 'CAMIÓN ARTICULADO FURGÓN'),
    ('14', 'CAMIÓN ARTICULADO BOTELLERO'),
    ('15', 'CAMIÓN ARTICULADO CISTERNA'),
    ('16', 'CAMIÓN ARTICULADO JAULA'),
    ('17', 'CAMIÓN ARTICULADO FRIGORÍFICO'),
    ('18', 'CAMIÓN ARTICULADO TALLER'),
    ('19', 'CAMIÓN ARTICULADO PARA CANTERA'),
    ('1A', 'CAMIÓN ARTICULADO VIVIENDA O CARAVANA'),
    ('1C', 'CAMIÓN ARTICULADO HORMIGONERA'),
    ('1D', 'CAMIÓN ARTICULADO VOLQUETE'),
    ('1E', 'CAMIÓN ARTICULADO GRÚA'),
    ('1F', 'CAMIÓN ARTICULADO CONTRA INCENDIOS'),
    ('20', 'FURGONETA'),
    ('21', 'FURGONETA MIXTA'),
    ('22', 'AMBULANCIA'),
    ('23', 'COCHE FÚNEBRE'),
    ('24', 'CAMIONETA'),
    ('25', 'TODO TERRENO'),
    ('30', 'AUTOBÚS'),
    ('31', 'AUTOBÚS ARTICULADO'),
    ('32', 'AUTOBÚS MIXTO'),
    ('33', 'BIBLIOBÚS'),
    ('34', 'AUTOBÚS LABORATORIO'),
    ('35', 'AUTOBÚS TALLER'),
    ('36', 'AUTOBÚS SANITARIO'),
    ('40', 'TURISMO'),
    ('50', 'MOTOCICLETA DE 2 RUEDAS SIN SIDECAR'),
    ('51', 'MOTOCICLETA CON SIDECAR'),
    ('52', 'MOTOCARRO'),
    ('53', 'AUTOMÓVIL DE 3 RUEDAS'),
    ('54', 'CUATRICICLO PESADO'),
    ('60', 'COCHE DE INVÁLIDO'),
    ('70', 'VEHÍCULO ESPECIAL'),
    ('71', 'PALA CARGADORA'),
    ('72', 'PALA EXCAVADORA'),
    ('73', 'CARRETILLA ELEVADORA'),
    ('74', 'MONIVELADORA'),
    ('75', 'COMPACTADORA'),
    ('76', 'APISONADORA'),
    ('77', 'GIROGRAVILLADORA'),
    ('78', 'MACHACADORA'),
    ('79', 'QUITANIEVES'),
    ('7A', 'VIVIENDA'),
    ('7B', 'BARREDORA'),
    ('7C', 'HORMIGONERA'),
    ('7D', 'VOLQUETE DE CANTERAS'),
    ('7E', 'GRÚA'),
    ('7F', 'SERVICIO CONTRA INCENDIOS'),
    ('7G', 'ASPIRADORA DE FANGOS'),
    ('7H', 'MOTOCULTOR'),
    ('7I', 'MAQUINARIA AGRÍCOLA AUTOMOTRIZ'),
    ('7J', 'PALA CARGADORA-RETROEXCAVADORA'),
    ('7K', 'TREN HASTA 160 PLAZAS'),
    ('80', 'TRACTOR'),
    ('81', 'TRACTOCAMIÓN'),
    ('82', 'TRACTOCARRO'),
    ('90', 'CICLOMOTOR DE 2 RUEDAS'),
    ('91', 'CICLOMOTOR DE 3 RUEDAS'),
    ('92', 'CUATRICICLO LIGERO'),
    ('EX', 'EXTRANJERO'),
    ('R0', 'REMOLQUE'),
    ('R1', 'REMOLQUE PLATAFORMA'),
    ('R2', 'REMOLQUE CAJA'),
    ('R3', 'REMOLQUE FURGÓN'),
    ('R4', 'REMOLQUE BOTELLERO'),
    ('R5', 'REMOLQUE CISTERNA'),
    ('R6', 'REMOLQUE JAULA'),
    ('R7', 'REMOLQUE FRIGORÍFICO'),
    ('R8', 'REMOLQUE TALLER'),
    ('R9', 'REMOLQUE PARA CANTERAS'),
    ('RA', 'REMOLQUE VIVIENDA O CARAVANA'),
    ('RB', 'REMOLQUE DE VIAJEROS O DE AUTOBÚS'),
    ('RC', 'REMOLQUE HORMIGONERA'),
    ('RD', 'REMOLQUE VOLQUETE DE CANTERA'),
    ('RE', 'REMOLQUE DE GRÚA'),
    ('RF', 'REMOLQUE CONTRA INCENDIOS'),
    ('RH', 'MAQ.AGRÍCOLA ARRASTRADA DE 2 EJES'),
    ('S0', 'SEMIRREMOLQUE'),
    ('S1', 'SEMIRREMOLQUE PLATAFORMA'),
    ('S2', 'SEMIRREMOLQUE CAJA'),
    ('S3', 'SEMIRREMOLQUE FURGÓN'),
    ('S4', 'SEMIRREMOLQUE BOTELLERO'),
    ('S5', 'SEMIRREMOLQUE CISTERNA'),
    ('S6', 'SEMIRREMOLQUE JAULA'),
    ('S7', 'SEMIRREMOLQUE FRIGORÍFICO'),
    ('S8', 'SEMIRREMOLQUE TALLER'),
    ('S9', 'SEMIRREMOLQUE CANTERA'),
    ('SA', 'SEMIRREMOLQUE VIVIENDA O CARAVANA'),
    ('SB', 'SEMIRREMOLQUE VIAJEROS O AUTOBÚS'),
    ('SC', 'SEMIRREMOLQUE HORMIGONERA'),
    ('SD', 'SEMIRREMOLQUE VOLQUETE DE CANTERA'),
    ('SE', 'SEMIRREMOLQUE GRÚA'),
    ('SF', 'SEMIRREMOLQUE CONTRA INCENDIOS'),
    ('SH', 'MAQ.AGRICOLA ARRASTRADA DE 1 EJE');
    
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_PROPULSION (COD_PROPULSION VARCHAR, DESCRIPCION VARCHAR);

INSERT INTO DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_PROPULSION VALUES
    ('0', 'Gasolina'),
    ('1', 'Diesel'),
    ('2', 'Eléctrico'),
    ('3', 'Otros'),
    ('4', 'Butano'),
    ('5', 'Solar'),
    ('6', 'Gas Licuado de Petróleo'),
    ('7', 'Gas Natural Comprimido'),
    ('8', 'Gas Natural Licuado'),
    ('9', 'Hidrógeno'),
    ('A', 'Biometano'),
    ('B', 'Etanol'),
    ('C', 'Biodiesel');
    
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_PROVINCIA_VEH (COD_PROVINCIA_VEH VARCHAR, DESCRIPCION VARCHAR);

INSERT INTO DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_PROVINCIA_VEH VALUES
    ('A', 'Alicante/Alacant'),
    ('AB', 'Albacete'),
    ('AL', 'Almería'),
    ('AV', 'Ávila'),
    ('B', 'Barcelona'),
    ('BA', 'Badajoz'),
    ('BI', 'Bizkaia'),
    ('BU', 'Burgos'),
    ('C', 'Coruña (A)'),
    ('CA', 'Cádiz'),
    ('CC', 'Cáceres'),
    ('CE', 'Ceuta'),
    ('CO', 'Córdoba'),
    ('CR', 'Ciudad Real'),
    ('CS', 'Castellón/Castelló'),
    ('CU', 'Cuenca'),
    ('DS', 'Desconocido'),
    ('EX', 'Extranjero'),
    ('GC', 'Palmas (Las)'),
    ('GI', 'Girona'),
    ('GR', 'Granada'),
    ('GU', 'Guadalajara'),
    ('H', 'Huelva'),
    ('HU', 'Huesca'),
    ('IB', 'Balears (Illes)'),
    ('J', 'Jaén'),
    ('L', 'Lleida'),
    ('LE', 'León'),
    ('LO', 'Rioja (La)'),
    ('LU', 'Lugo'),
    ('M', 'Madrid'),
    ('MA', 'Málaga'),
    ('ML', 'Melilla'),
    ('MU', 'Murcia'),
    ('NA', 'Navarra'),
    ('O', 'Asturias'),
    ('OU', 'Ourense'),
    ('P', 'Palencia'),
    ('PO', 'Pontevedra'),
    ('S', 'Cantabria'),
    ('SA', 'Salamanca'),
    ('SE', 'Sevilla'),
    ('SG', 'Segovia'),
    ('SO', 'Soria'),
    ('SS', 'Gipuzkoa'),
    ('T', 'Tarragona'),
    ('TE', 'Teruel'),
    ('TF', 'Santa Cruz de Tenerife'),
    ('TO', 'Toledo'),
    ('V', 'Valencia/València'),
    ('VA', 'Valladolid'),
    ('VI', 'Araba/Álava'),
    ('Z', 'Zaragoza'),
    ('ZA', 'Zamora');
    
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_PROVINCIA_MAT (COD_PROVINCIA_MAT VARCHAR, DESCRIPCION VARCHAR);

INSERT INTO DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_PROVINCIA_MAT VALUES
    ('A', 'Alicante/Alacant'),
    ('AB', 'Albacete'),
    ('AL', 'Almería'),
    ('AV', 'Ávila'),
    ('B', 'Barcelona'),
    ('BA', 'Badajoz'),
    ('BI', 'Bizkaia'),
    ('BU', 'Burgos'),
    ('C', 'Coruña (A)'),
    ('CA', 'Cádiz'),
    ('CC', 'Cáceres'),
    ('CE', 'Ceuta'),
    ('CO', 'Córdoba'),
    ('CR', 'Ciudad Real'),
    ('CS', 'Castellón/Castelló'),
    ('CU', 'Cuenca'),
    ('DS', 'Desconocido'),
    ('EX', 'Extranjero'),
    ('GC', 'Palmas (Las)'),
    ('GI', 'Girona'),
    ('GR', 'Granada'),
    ('GU', 'Guadalajara'),
    ('H', 'Huelva'),
    ('HU', 'Huesca'),
    ('IB', 'Balears (Illes)'),
    ('J', 'Jaén'),
    ('L', 'Lleida'),
    ('LE', 'León'),
    ('LO', 'Rioja (La)'),
    ('LU', 'Lugo'),
    ('M', 'Madrid'),
    ('MA', 'Málaga'),
    ('ML', 'Melilla'),
    ('MU', 'Murcia'),
    ('NA', 'Navarra'),
    ('O', 'Asturias'),
    ('OU', 'Ourense'),
    ('P', 'Palencia'),
    ('PO', 'Pontevedra'),
    ('S', 'Cantabria'),
    ('SA', 'Salamanca'),
    ('SE', 'Sevilla'),
    ('SG', 'Segovia'),
    ('SO', 'Soria'),
    ('SS', 'Gipuzkoa'),
    ('T', 'Tarragona'),
    ('TE', 'Teruel'),
    ('TF', 'Santa Cruz de Tenerife'),
    ('TO', 'Toledo'),
    ('V', 'Valencia/València'),
    ('VA', 'Valladolid'),
    ('VI', 'Araba/Álava'),
    ('Z', 'Zaragoza'),
    ('ZA', 'Zamora');
    
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_CLAVE_TRAMITE (CLAVE_TRAMITE VARCHAR, DESCRIPCION VARCHAR);

INSERT INTO DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_CLAVE_TRAMITE VALUES
    ('1', 'Matriculación ordinaria y de ciclomotores'),
    ('2', 'Transferencia'),
    ('3', 'Baja definitiva (excluidos Plan Renove, Baja por exportación y Tránsito comunitario)'),
    ('4', 'Baja definitiva por Plan Renove'),
    ('5', 'Rematriculación'),
    ('6', 'Baja temporal'),
    ('7', 'Baja definitiva por Exportación y por Tránsito comunitario'),
    ('8', 'Matriculación vehículo especial'),
    ('9', 'Matriculación temporal'),
    ('A', 'Prorroga matricula temporal'),
    ('B', 'Paso de matrícula temporal a definitiva');
    
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_SERVICIO (SERVICIO VARCHAR, DESCRIPCION VARCHAR);

INSERT INTO DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_SERVICIO VALUES
    ('A01', 'Público PUBL-Alquiler sin conductor'),
    ('A02', 'Público PUBL-Alquiler con conductor'),
    ('A03', 'Público PUBL-Aprendizaje de conducción'),
    ('A04', 'Público PUBL-Taxi'),
    ('A05', 'Público PUBL-Auxilio en carretera'),
    ('A07', 'Público PUBL-Ambulancia'),
    ('A08', 'Público PUBL-Funerario'),
    ('A09', 'Particular PART-Obras'),
    ('A10', 'Público PUBL-Mercancías peligrosas'),
    ('A11', 'Público PUBL-Basurero'),
    ('A12', 'Público PUBL-Transporte escolar'),
    ('A13', 'Público PUBL-Policía'),
    ('A14', 'Público PUBL-Bomberos'),
    ('A15', 'Público PUBL-Protección civil y salvamento'),
    ('A16', 'Público PUBL-Defensa'),
    ('A18', 'Público PUBL-Actividad económica'),
    ('A20', 'Público PUBL-Mercancías perecederas'),
    ('B00', 'Particular PART-Sin especificar'),
    ('B06', 'Particular PART-Agrícola'),
    ('B07', 'Particular PART-'),
    ('B09', 'Particular PART-Obras'),
    ('B17', 'Particular PART-Vivienda'),
    ('B18', 'Público PART-Actividad económica'),
    ('B19', 'Particular PART-Recreativo'),
    ('B21', 'Particular PART-Vehículo para ferias');
    
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_IND_BAJA_DEF (IND_BAJA_DEF VARCHAR, DESCRIPCION VARCHAR);

INSERT INTO DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_IND_BAJA_DEF VALUES
    ('0', 'Desguace'),
    ('1', 'Agotamiento'),
    ('2', 'Antigüedad'),
    ('3', 'Renovación del parque'),
    ('4', 'Otros motivos'),
    ('5', 'R.D.L 4/1994 , R.D.L 10/1994 , R.D.L 4/1997'),
    ('7', 'Voluntaria'),
    ('8', 'Exportación'),
    ('9', 'Transito comunitario'),
    ('A', 'De oficio por abandono'),
    ('B', 'De oficio por seguridad'),
    ('C', 'Por Tratamiento Residual');
    
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_CATEGORIA_VEHICULO_ELECTRICO (CATEGORIA_VEHICULO_ELECTRICO VARCHAR, DESCRIPCION VARCHAR);

INSERT INTO DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_CATEGORIA_VEHICULO_ELECTRICO VALUES
    ('PHEV', 'Eléctrico enchufable'),
    ('REEV', 'Eléctrico de autonomía extendida'),
    ('HEV', 'Eléctrico híbrido'),
    ('BEV', 'Eléctrico de batería');

In [ ]:
SELECT * FROM DEMO_VENTA_VEHICULOS.REFINED.FACT_MATRICULACIONES LIMIT 10;

# Crear tabla final
Crear una tabla final donde se utilicen la descripción de las distintas variables en lugar de sus códigos y se filtre por matrículaciones ordinarias.

In [ ]:
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.CURATED.MATRICULACIONES_VEHICULOS 
AS SELECT
    TO_DATE(t1.FECHA_MATRICULA, 'DDMMYYYY') AS FECHA_DE_MATRICULACION,
    t2.DESCRIPCION AS CLASE_MATRICULA,
    t1.MARCA_ITV AS MARCA,
    t1.MODELO_ITV AS MODELO,
    t3.DESCRIPCION AS PROCEDENCIA_VEHICULO,
    t1.BASTIDOR_ITV AS CODIGO_BASTIDOR,
    t4.DESCRIPCION AS TIPO_VEHICULO,
    t5.DESCRIPCION AS TIPO_PROPULSION,
    t1.CILINDRADA_ITV::NUMBER AS CILINDRADA,
    t1.POTENCIA_ITV::NUMBER AS POTENCIA,
    IFF(t1.TARA<>'******',t1.TARA,0)::NUMBER AS TARA,
    t1.PESO_MAX::NUMBER AS PESO_MAXIMO,
    t1.NUM_PLAZAS::INTEGER AS PLAZAS,
    t6.DESCRIPCION AS PROVINCIA_MATRICULACION,
    t7.DESCRIPCION AS TRAMITE,
    t1.CODIGO_POSTAL AS CODIGO_POSTAL,
    IFF(IND_NUEVO_USADO='N',TRUE,FALSE) AS VEHICULO_NUEVO,
    IFF(PERSONA_FISICA_JURIDICA='D',TRUE,FALSE) AS PERSONA_FISICA,
    t8.DESCRIPCION AS TIPO_SERVICIO,
    t1.COD_MUNICIPIO_INE_VEH AS CODIGO_MUNICIPIO,
    t1.MUNICIPIO AS MUNICIPIO,
    IFF(t1.KW_ITV<>'*******',t1.KW_ITV,NULL)::NUMBER AS POTENCIA_MAXIMA,
    IFF(t1.RENTING='N',FALSE,TRUE)::BOOLEAN AS RENTING,
    t9.DESCRIPCION AS CATEGORIA_VEHICULO_ELECTRICO
FROM
    DEMO_VENTA_VEHICULOS.REFINED.FACT_MATRICULACIONES t1
    LEFT JOIN DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_CLASE_MAT t2 ON t1.COD_CLASE_MAT = t2.COD_CLASE_MAT
    LEFT JOIN DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_PROCEDENCIA t3 ON t1.COD_PROCEDENCIA_ITV = t3.COD_PROCEDENCIA
    LEFT JOIN DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_TIPO t4 ON t1.COD_TIPO = t4.COD_TIPO
    LEFT JOIN DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_PROPULSION t5 ON t1.COD_PROPULSION_ITV = t5.COD_PROPULSION
    LEFT JOIN DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_COD_PROVINCIA_MAT t6 ON t1.COD_PROVINCITA_MAT = t6.COD_PROVINCIA_MAT
    LEFT JOIN DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_CLAVE_TRAMITE t7 ON t1.CLAVE_TRAMITE = t7.DESCRIPCION
    LEFT JOIN DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_SERVICIO t8 ON t1.SERVICIO = t8.SERVICIO
    LEFT JOIN DEMO_VENTA_VEHICULOS.REFINED.DIM_MATRICULACIONES_CATEGORIA_VEHICULO_ELECTRICO t9 ON  t1.CATEGORIA_VEHICULO_ELECTRICO = t9.CATEGORIA_VEHICULO_ELECTRICO
        WHERE t1.CLAVE_TRAMITE = '1' AND t1.COD_CLASE_MAT = '0' ;

    
SELECT * 
    FROM DEMO_VENTA_VEHICULOS.CURATED.MATRICULACIONES_VEHICULOS
    LIMIT 100;

# Crear tablas para muestra de datos de CRM
En esta sección se crearán las tablas de Oportunidades, Clientes, Vehiculos y Tareas del CRM.
Además, se cargarán los datos desde un Bucket de S3.

In [ ]:
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.RAW.CRM_CLIENTES (
	JSON VARIANT
);

-- Insertar datos de S3 en tabla RAW
COPY INTO DEMO_VENTA_VEHICULOS.RAW.CRM_CLIENTES
FROM 's3://crmdatosdeejemplovehiculosspn/customers.json'
FILE_FORMAT = ( TYPE = JSON , STRIP_OUTER_ARRAY = TRUE )
ON_ERROR = 'skip_file';


CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.RAW.CRM_OPORTUNIDADES (
	JSON VARIANT
);

-- Insertar datos de S3 en tabla RAW
COPY INTO DEMO_VENTA_VEHICULOS.RAW.CRM_OPORTUNIDADES
FROM 's3://crmdatosdeejemplovehiculosspn/opportunities.json'
FILE_FORMAT = ( TYPE = JSON , STRIP_OUTER_ARRAY = TRUE )
ON_ERROR = 'skip_file';

CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.RAW.CRM_TAREAS (
	JSON VARIANT
);

-- Insertar datos de S3 en tabla RAW
COPY INTO DEMO_VENTA_VEHICULOS.RAW.CRM_TAREAS
FROM 's3://crmdatosdeejemplovehiculosspn/tasks.json'
FILE_FORMAT = ( TYPE = JSON , STRIP_OUTER_ARRAY = TRUE )
ON_ERROR = 'skip_file';

CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.RAW.CRM_VEHICULOS (
	JSON VARIANT
);

-- Insertar datos de S3 en tabla RAW
COPY INTO DEMO_VENTA_VEHICULOS.RAW.CRM_VEHICULOS
FROM 's3://crmdatosdeejemplovehiculosspn/vehicles.json'
FILE_FORMAT = ( TYPE = JSON , STRIP_OUTER_ARRAY = TRUE )
ON_ERROR = 'skip_file';

CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.CRM_CLIENTES (
	APELLIDOS VARCHAR(16777216),
	CORREOELECTRONICO VARCHAR(16777216),
	CUSTOMERID VARCHAR(16777216),
	EDADCLIENTE NUMBER(38,0),
	NOMBRE VARCHAR(16777216),
	PUNTUACIONCREDITO NUMBER(38,0),
	TELEFONO VARCHAR(16777216),
	UBICACION VARCHAR(16777216),
	constraint PK_CLIENTES primary key (CUSTOMERID)
) AS 
SELECT 
    JSON:apellidos::VARCHAR,
    JSON:correoElectronico::VARCHAR,
    JSON:customerId::VARCHAR,
    JSON:edadCliente::INTEGER,
    JSON:nombre::VARCHAR,
    JSON:puntuacionCredito::INTEGER,
    JSON:telefono::VARCHAR,
    JSON:ubicacion::VARCHAR
FROM DEMO_VENTA_VEHICULOS.RAW.CRM_CLIENTES
;

CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.CRM_OPORTUNIDADES (
	COTIZACIONENVIADA BOOLEAN,
	CUSTOMERID VARCHAR(16777216),
	ETAPAVENTA VARCHAR(16777216),
	FECHACREACION DATE,
	FECHADECISIONESPERADA DATE,
	FECHAULTIMAACTIVIDAD DATE,
	IDOPORTUNIDAD VARCHAR(16777216),
	NOTASSEGUIMIENTO VARCHAR(16777216),
	ORIGENPROSPECTO VARCHAR(16777216),
	PROBABILIDAD NUMBER(38,0),
	PRODUCTID VARCHAR(16777216),
	PROXIMOSPASOS VARCHAR(16777216),
	PRUEBAMANEJOPEND BOOLEAN,
	RANGOPRESUPUESTO VARCHAR(16777216),
	REPRESENTANTEVENTAS VARCHAR(16777216),
	RESUMENOPORTUNIDAD VARCHAR(16777216),
	TIPOFINANCIACION VARCHAR(16777216),
	TIPOVENTA VARCHAR(16777216),
	VALORNEGOCIO NUMBER(38,0),
	VEHICULOENTREGA VARCHAR(16777216),
	constraint PK_OPORTUNIDADES primary key (IDOPORTUNIDAD)
)
AS 
SELECT 
    JSON:cotizacionEnviada::BOOLEAN,
    JSON:customerId::VARCHAR,
    JSON:etapaVenta::VARCHAR,
    JSON:fechaCreacion::DATE,
    JSON:fechaDecisionEsperada::DATE,
    JSON:fechaUltimaActividad::DATE,
    JSON:idOportunidad::VARCHAR,
    JSON:notasSeguimiento::VARCHAR,
    JSON:origenProspecto::VARCHAR,
    JSON:probabilidad::INTEGER,
    JSON:productoId::VARCHAR,
    JSON:proximosPasos::VARCHAR,
    JSON:pruebaManejoPend::BOOLEAN,
    JSON:rangoPresupuesto::VARCHAR,
    JSON:representanteVentas::VARCHAR,
    JSON:resumenOportunidad::VARCHAR,
    JSON:tipoFinanciacion::VARCHAR,
    JSON:tipoVenta::VARCHAR,
    JSON:valorNegocio::INTEGER,
    JSON:vehiculoEntrega::VARCHAR
FROM DEMO_VENTA_VEHICULOS.RAW.CRM_OPORTUNIDADES
;

CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.CRM_TAREAS (
	CUSTOMERID VARCHAR(16777216),
	ESTADO VARCHAR(16777216),
	FECHA_TAREA DATE,
	IDOPORTUNIDAD VARCHAR(16777216),
	TAREA VARCHAR(16777216),
	TASKID VARCHAR(16777216),
	constraint PK_TAREAS primary key (TASKID)
)
AS SELECT
    JSON:customerId::VARCHAR,
    JSON:estado::VARCHAR,
    JSON:fecha_tarea::DATE,
    JSON:idOportunidad::VARCHAR,
    JSON:tarea::VARCHAR,
    JSON:taskId::VARCHAR
FROM DEMO_VENTA_VEHICULOS.RAW.CRM_TAREAS
;

CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.REFINED.CRM_VEHICULOS (
	MARCAVEHICULO VARCHAR(16777216),
	MODELOVEHICULO VARCHAR(16777216),
	PRODUCTID VARCHAR(16777216),
	TIPOVEHICULO VARCHAR(16777216),
	constraint PK_VEHICULOS primary key (PRODUCTID)
)
AS SELECT
    JSON:marcaVehiculo::VARCHAR,
    JSON:modeloVehiculo::VARCHAR,
    JSON:productId::VARCHAR,
    JSON:tipoVehiculo::VARCHAR
FROM DEMO_VENTA_VEHICULOS.RAW.CRM_VEHICULOS;

In [ ]:
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.CURATED.CRM_OPORTUNIDADES (
	id_oportunidad VARCHAR,
	id_cliente VARCHAR,
	id_producto VARCHAR,
	cotizacion_enviada BOOLEAN,
	etapa VARCHAR,
	fecha_creacion DATE,
	fecha_decision_esperada DATE,
	fecha_ultima_oportunidad DATE,
	notas_seguimiento VARCHAR,
	origen_oportunidad VARCHAR,
	probabilidad NUMBER,
	proximos_pasos VARCHAR,
	prueba_pendiente BOOLEAN,
	rango_presupuesto VARCHAR,
	representante_ventas VARCHAR,
	resumen_oportunidad VARCHAR,
	tipo_financiacion VARCHAR,
	tipo_venta VARCHAR,
	valor_estimado NUMBER,
	vehiculo_entrega VARCHAR,
    constraint PK_OPORTUNIDADES primary key (id_oportunidad)
)
AS SELECT 
    IDOPORTUNIDAD,
    CUSTOMERID,
  	PRODUCTID,
    COTIZACIONENVIADA,
	ETAPAVENTA,
	FECHACREACION,
	FECHADECISIONESPERADA,
	FECHAULTIMAACTIVIDAD,
	NOTASSEGUIMIENTO,
	ORIGENPROSPECTO,
	PROBABILIDAD,
	PROXIMOSPASOS,
	PRUEBAMANEJOPEND,
	RANGOPRESUPUESTO,
	REPRESENTANTEVENTAS,
    RESUMENOPORTUNIDAD,
	TIPOFINANCIACION,
	TIPOVENTA,
	VALORNEGOCIO,
	VEHICULOENTREGA,
FROM DEMO_VENTA_VEHICULOS.REFINED.CRM_OPORTUNIDADES;

CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.CURATED.CRM_TAREAS (
	ID_TAREA VARCHAR,
	ID_CLIENTE VARCHAR,
    ID_OPORTUNIDAD VARCHAR,
    ESTADO VARCHAR,
	FECHA_TAREA DATE,
	TAREA VARCHAR,
    constraint PK_TAREAS primary key (ID_TAREA)
)
AS SELECT
	TASKID,
    CUSTOMERID,
    IDOPORTUNIDAD,
	ESTADO,
	FECHA_TAREA,
    TAREA
FROM DEMO_VENTA_VEHICULOS.REFINED.CRM_TAREAS;

CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.CURATED.CRM_VEHICULOS (
	ID_PRODUCTO VARCHAR,
    MARCA_VEHICULO VARCHAR,
    MODELO_VEHICULO VARCHAR,
    TIPO_VEHICULO VARCHAR,
    constraint PK_TAREAS primary key (ID_PRODUCTO)
)
AS SELECT
    PRODUCTID,
	MARCAVEHICULO,
	MODELOVEHICULO,
	TIPOVEHICULO
FROM DEMO_VENTA_VEHICULOS.REFINED.CRM_VEHICULOS;
;

CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.CURATED.CRM_CLIENTES (
	ID_CLIENTE VARCHAR,
    NOMBRE VARCHAR,
    APELLIDOS VARCHAR,
	CORREO_ELECTRONICO VARCHAR,
	EDAD_CLIENTE INTEGER,
	PUNTUACION_CREDITO INTEGER,
	TELEFONO VARCHAR,
	UBICACION VARCHAR,
    constraint PK_CLIENTES primary key (ID_CLIENTE)
) AS 
SELECT 
    CUSTOMERID,
    NOMBRE,
    APELLIDOS,
    CORREOELECTRONICO,
    EDADCLIENTE,
    PUNTUACIONCREDITO,
    TELEFONO,
    UBICACION
FROM DEMO_VENTA_VEHICULOS.REFINED.CRM_CLIENTES

# Introducing Cortex AISQL: Reimagining SQL into AI Query Language for Multimodal Data
![AISQL Capabilities](https://publish-p57963-e462109.adobeaemcloud.com/adobe/dynamicmedia/deliver/dm-aid--b5888b22-4a2c-4508-b18d-b5a2fe33508a/screenshot-2025-05-30-at-8.32.29%25E2%2580%25AFam.png?quality=85&preferwebp=true&width=1920)


In [ ]:
SELECT 
    ID_OPORTUNIDAD,
    RESUMEN_OPORTUNIDAD,
    AI_CLASSIFY(RESUMEN_OPORTUNIDAD,
        [
            'Garantía',
            'Comodidad',
            'Precio',
            'Rendimiento',
            'Versatilidad',
            'Eléctrico',
            'Bajo mantenimiento',
            'Seguridad',
            'Espacio de carga',
            'Consumo',
            'Tecnología',
            'Automático',
            'Estilo'
        ]
    )
FROM DEMO_VENTA_VEHICULOS.CURATED.CRM_OPORTUNIDADES LIMIT 20;

In [ ]:
SELECT 
    ID_OPORTUNIDAD,
    RESUMEN_OPORTUNIDAD,
    AI_CLASSIFY(RESUMEN_OPORTUNIDAD,
        [
            'Garantía',
            'Comodidad',
            'Precio',
            'Rendimiento',
            'Versatilidad',
            'Eléctrico',
            'Bajo mantenimiento',
            'Seguridad',
            'Espacio de carga',
            'Consumo',
            'Tecnología',
            'Automático',
            'Estilo'
        ],
        {'output_mode': 'multi'}
    )
FROM DEMO_VENTA_VEHICULOS.CURATED.CRM_OPORTUNIDADES LIMIT 20;

In [ ]:
SELECT 
    ID_OPORTUNIDAD,
    RESUMEN_OPORTUNIDAD,
    PARSE_JSON(TO_VARCHAR(
    AI_CLASSIFY(RESUMEN_OPORTUNIDAD,
        [
            {'label': 'Garantía',
              'description': 'Cobertura extendida y protección a largo plazo del vehículo'},
            {'label': 'Comodidad',
            'description': 'Confort en asientos, climatización y experiencia de conducción'},
            {'label': 'Precio',
            'description': 'Relación calidad-precio y ajuste al presupuesto disponible'},
            {'label': 'Rendimiento',
            'description': 'Potencia del motor, aceleración y capacidad de respuesta'},
            {'label': 'Versatilidad',
            'description': 'Adaptabilidad para diferentes usos y configuraciones del habitáculo'},
            {'label': 'Eléctrico',
            'description': 'Tecnología híbrida o 100% eléctrica, sostenibilidad ambiental'},
            {'label': 'Bajo mantenimiento',
            'description': 'Costes reducidos de reparación y mantenimiento preventivo'},
            {'label': 'Seguridad',
            'description': 'Sistemas avanzados de protección, airbags y asistencia a la conducción'},
            {'label': 'Espacio de carga',
            'description': 'Capacidad del maletero y área de almacenamiento de equipaje'},
            {'label': 'Consumo',
            'description': 'Eficiencia en el gasto de combustible, ahorro económico'},
            {'label': 'Tecnología',
            'description': 'Conectividad, pantallas táctiles y sistemas de infoentretenimiento'},
            {'label': 'Automático',
            'description': 'Transmisión automática para facilidad de conducción'},
            {'label': 'Estilo',
            'description': 'Diseño exterior e interior, estética y prestigio de marca'}
        ],
        {'output_mode': 'multi',
        'task_description': 'Indica las características más importantes para el cliente interesado en comprar un vehículo en base a este resumen de la oportunidad. No incluyas más de tres características, centrándote siempre en las que parezcan más relevantes por el resumen de la oportunidad.'}
    ))):labels::VARCHAR AS CARACTERISTICAS_CLAVE,
    PROXIMOS_PASOS,
    AI_TRANSLATE(PROXIMOS_PASOS,'es','fr') AS NEXT_STEPS_FR
FROM DEMO_VENTA_VEHICULOS.CURATED.CRM_OPORTUNIDADES LIMIT 20;

In [ ]:
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.CURATED.CRM_OPORTUNIDADES_ENRIQUECIDA AS
SELECT 
    *,
    PARSE_JSON(TO_VARCHAR(
    AI_CLASSIFY(RESUMEN_OPORTUNIDAD,
        [
            {'label': 'Garantía',
              'description': 'Cobertura extendida y protección a largo plazo del vehículo'},
            {'label': 'Comodidad',
            'description': 'Confort en asientos, climatización y experiencia de conducción'},
            {'label': 'Precio',
            'description': 'Relación calidad-precio y ajuste al presupuesto disponible'},
            {'label': 'Rendimiento',
            'description': 'Potencia del motor, aceleración y capacidad de respuesta'},
            {'label': 'Versatilidad',
            'description': 'Adaptabilidad para diferentes usos y configuraciones del habitáculo'},
            {'label': 'Eléctrico',
            'description': 'Tecnología híbrida o 100% eléctrica, sostenibilidad ambiental'},
            {'label': 'Bajo mantenimiento',
            'description': 'Costes reducidos de reparación y mantenimiento preventivo'},
            {'label': 'Seguridad',
            'description': 'Sistemas avanzados de protección, airbags y asistencia a la conducción'},
            {'label': 'Espacio de carga',
            'description': 'Capacidad del maletero y área de almacenamiento de equipaje'},
            {'label': 'Consumo',
            'description': 'Eficiencia en el gasto de combustible, ahorro económico'},
            {'label': 'Tecnología',
            'description': 'Conectividad, pantallas táctiles y sistemas de infoentretenimiento'},
            {'label': 'Automático',
            'description': 'Transmisión automática para facilidad de conducción'},
            {'label': 'Estilo',
            'description': 'Diseño exterior e interior, estética y prestigio de marca'}
        ],
        {'output_mode': 'multi',
        'task_description': 'Indica las características más importantes para el cliente interesado en comprar un vehículo en base a este resumen de la oportunidad. No incluyas más de tres características, centrándote siempre en las que parezcan más relevantes por el resumen de la oportunidad.'}
    ))):labels::VARCHAR AS CARACTERISTICAS_CLAVE,
FROM DEMO_VENTA_VEHICULOS.CURATED.CRM_OPORTUNIDADES;

In [ ]:
ALTER TABLE DEMO_VENTA_VEHICULOS.CURATED.CRM_OPORTUNIDADES_ENRIQUECIDA 
ADD PRIMARY KEY (ID_OPORTUNIDAD);

# Cortex Search 
Get up and running with a hybrid (vector and keyword) search engine on your text data in minutes, without having to worry about embedding, infrastructure maintenance, search quality parameter tuning, or ongoing index refreshes.
![Snowflake Search](https://docs.snowflake.com/en/_images/cortex-search-rag.png)


In [ ]:
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.RAW.MANUALES_PDF AS
SELECT
    RELATIVE_PATH,
    SIZE,
    TO_VARCHAR(
        AI_PARSE_DOCUMENT(   TO_FILE('@DEMO_VENTA_VEHICULOS.RAW.MANUALES',RELATIVE_PATH),{'mode': 'LAYOUT'})) AS RAW_TEXT
FROM DIRECTORY('@DEMO_VENTA_VEHICULOS.RAW.MANUALES');

In [ ]:
CREATE OR REPLACE TABLE DEMO_VENTA_VEHICULOS.RAW.RAW_MANUALES_CHUNKS AS
SELECT
relative_path AS DOCUMENTO,
--REGEXP_SUBSTR(relative_path,'(.+)/.+.pdf',1,1,'i',1) as MARCA,
--REGEXP_SUBSTR(relative_path, '/\\w+\\s+([^\\s]+)', 1, 1, 'i', 1) as MODELO,
SPLIT_PART(relative_path, '_', 1) AS MARCA,
    SPLIT_PART(relative_path, '_', 2) AS MODELO,
BUILD_SCOPED_FILE_URL(@DEMO_VENTA_VEHICULOS.RAW.MANUALES, relative_path) AS file_url,
C.value::VARCHAR as chunk
FROM
DEMO_VENTA_VEHICULOS.RAW.MANUALES_PDF,
LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
RAW_TEXT,'none',2000,300
)) c;

In [ ]:
select * from demo_venta_vehiculos.raw.raw_manuales_chunks limit 100;

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE DEMO_VENTA_VEHICULOS.CURATED.SEARCH_SERVICE_MANUALES_VEHICULOS
    ON chunk
    ATTRIBUTES marca,modelo,documento
    WAREHOUSE = cortex_analyst_wh
    TARGET_LAG = '1 hour'
    AS (
    SELECT
        chunk,
        documento,
        file_url,
        marca,
        modelo
    FROM DEMO_VENTA_VEHICULOS.RAW.RAW_MANUALES_CHUNKS
    );

In [ ]:
CREATE TABLE DEMO_VENTA_VEHICULOS.RAW.COMPETENCIA
(
MODELO VARCHAR PRIMARY KEY,
COMPETENCIA VARCHAR
);

INSERT INTO DEMO_VENTA_VEHICULOS.RAW.COMPETENCIA 
SELECT * FROM VALUES
  ('Qashqai', '{"COMPETIDOR_QASHQAI_1": {"MARCA": "KIA","MODELO": "SPORTAGE"},"COMPETIDOR_QASHQAI_2": {"MARCA": "HYUNDAI","MODELO": "TUCSON"}}'),
  ('Ariya', '{"COMPETIDOR_ARIYA_1": {"MARCA": "KIA","MODELO": "EV6"},"COMPETIDOR_ARIYA_2": {"MARCA": "TESLA","MODELO": "MODEL Y"}}'),
  ('Juke', '{"COMPETIDOR_JUKE_1": {"MARCA": "SEAT","MODELO": "ARONA"},"COMPETIDOR_JUKE_2": {"MARCA": "HYUNDAI","MODELO": "KONA"}}'),
  ('Micra', '{"COMPETIDOR_MICRA_1": {"MARCA": "SEAT","MODELO": "IBIZA"},"COMPETIDOR_MICRA_2": {"MARCA": "RENAULT","MODELO": "CLIO"}}'),
  ('X-Trail', '{"COMPETIDOR_X-TRAIL_1": {"MARCA": "RENAULT","MODELO": "ESPACE"},"COMPETIDOR_X-TRAIL_2": {"MARCA": "MITSUBISHI","MODELO": "OUTLANDER"}}')
AS t(modelo, competencia);

CREATE CORTEX SEARCH SERVICE DEMO_VENTA_VEHICULOS.CURATED.search_service_competencia
  ON competencia
  ATTRIBUTES modelo
  WAREHOUSE = ai_xs
  TARGET_LAG = '1 hour'
  INITIALIZE = ON_CREATE
  AS select modelo, competencia from DEMO_VENTA_VEHICULOS.RAW.competencia;